# Проект по SQL

Компания приобрела крупный сервис для чтения книг по подписке. База данных сервиса содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Цель** - проанализировать базу данных., для определения ценностностого предложения для нового продукта

**Задачи:**

1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [1]:
#Импортируем библиотеки 
import pandas as pd 
from sqlalchemy import text, create_engine  

#Устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя 
             'pwd': 'Sdf4$2;d-d30pp', # пароль 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 'port': 6432, # порт подключения 
             'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)  

#Cохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})  

**Обзор данных**

In [2]:
#Вывод значений
query = '''

SELECT *
FROM books
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица **books** содержит информацию о книгах (идентификатор книги, идентификатор автора, наименование книги, количество страниц, дату публикации и идентификатор издателя)

In [3]:
#Вывод значений
query = '''

SELECT *
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица **authors** содержит информацию об авторах книг (идентификатор автора, имя автора)

In [4]:
#Вывод значений
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица **publishers** содержит информацию об издательствах (идентификатор издательства, название издательства)

In [5]:
#Вывод значений
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица **ratings** содержит информацию о пользовательских оценках книг (идентификатор оценки, идентификатор книги, имя пользователя, оценка)

In [6]:
#Вывод значений
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица **reviews** содержит информацию о пользовательских обзорах (идентификатор обзора, идентификатор книги, имя пользователя, текст обзора)

# Задания

**Количество книг после 1 января 2000 года**

In [7]:
query = '''

SELECT COUNT (publication_date)
FROM books
WHERE publication_date > '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


**Количество обзоров и средняя оценка книги**

In [8]:
query = '''

SELECT b.book_id,
       b.title,
       COUNT(DISTINCT rev.review_id) as count_reviews,
       AVG(rat.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id

GROUP BY b.book_id, b.title

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_reviews,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


**Издательство, которое выпустило наибольшее число книг толще 50 страниц** 

In [9]:
query = '''

SELECT publisher,
       COUNT (book_id) AS count_books
FROM publishers AS pub
INNER JOIN books AS b ON pub.publisher_id = b.publisher_id 
WHERE b.num_pages > 50 
GROUP BY publisher
ORDER BY count_books DESC
LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


**Автор с самой высокой средней оценкой книг**

In [10]:
query = '''

SELECT author,
       AVG(rating) AS avg_rating
FROM books
LEFT JOIN authors ON books.author_id =authors.author_id
LEFT JOIN ratings ON books.book_id=ratings.book_id
WHERE books.book_id IN (SELECT b.book_id
                  FROM books AS b
                  LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
                  GROUP BY b.book_id
                  HAVING COUNT(rat.rating_id) >=50)
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1

'''
    
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [11]:
query = '''
SELECT COUNT(DISTINCT rev.review_id)/COUNT(DISTINCT rev.username) AS avg_reviews
FROM ratings AS rat
INNER JOIN reviews AS rev ON rat.book_id=rev.book_id
WHERE rev.username IN (SELECT username
                   FROM ratings
                   GROUP BY username
                   HAVING COUNT (rating_id) > 48)


'''
    
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24


# Вывод

В результате проведенного анализа получены следующие ответы на поставленные задачи:

1. Количество книг, вышедших после 1 января 2000 года - 819;
2. Количество обзоров и средняя оценка книги - рассчитали среднюю оценку и количество обзоров, данные отобразили в отдельных столбцах;
3. Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books (42 книги);
4. Автора с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (рейтинг 4.287097)
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора